# GDMA Project
Author: Julian Schelb (1069967)

In [1]:
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Connection to the database instance

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "subatomic-shrank-Respond"))
database_name = "cddb"
session = driver.session(database = database_name)

### Task 4: Searching and Ranking

Implement a simple search engine that enables search by artist, album and
song name/title. The results must be ranked based on importance. It is up to
you to come up with how the importance of each result is computed and you
must justify your decision (it goes without saying that you need to come up
with a meaningful definition). However, the importance should ideally take into
account user preferences/likes. As such, this task is split in two parts:

**1. Write a Cypher query that adds a relationship :LIKES between a node with**

label :User and an artist, album, or song. Every user should be identified
just by a numerical userID (no more information is necessary). If a user
already exists in the system, no additional node should be added. After
coming up with the necessary Cypher query, add a significant number of
users and likes.

**2. Implement a simple Python function that has the following arguments:**

- the userID of the user submitting the search (the user ID may not
exist in the database),
- a string that contains one or more keywords for the search, and
- an optional argument that indicates whether the search is on all or
a specific field, i.e., artist, album, song.
The search must return exactly 10 results.

Python must only be used to call the database. You should not write any
code in Python that implements functionality necessary for the task. However,
submitting multiple queries in the same function call is allowed. Also, for this
task of the project, you are not only allowed but also encouraged to use functions
from the GDS library of Neo4j. Hence, before making any decisions, have a
careful look at the available functions. Again, you have to justify the use of any
function that you employ

#### Adding Example Users

**Function to create Example User:**

In [53]:
def createExampleUser(user_id = 1, genre = "rock", limit = 50):
    
    ####### DELETE USER NODE #######
    query1 = """
    MATCH (u:User)
    WHERE u.id = $user_id
    DETACH DELETE u
    """
    
    ####### CREATE USER NODE #######
    query2 = """
    MERGE (u:User {id:  $user_id})
    RETURN u.id as user_id
    """
    
    ####### LINK TO LIKED SONGS #######
    query3 = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(s:Song)
    WHERE g.genre = $genre
    WITH s 
    LIMIT $limit
    MATCH (u:User)
    WHERE u.id = $user_id
    MERGE (u)-[r3:LIKES]->(s)
    """
    
    ####### LINK TO LIKED ARTISTS #######
    query4 = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(t:Artist)
    WHERE g.genre = $genre
    WITH t 
    LIMIT $limit
    MATCH (u:User)
    WHERE u.id = $user_id
    MERGE (u)-[r3:LIKES]->(t)
    """
    
    ####### LINK TO LIKED ALBUMS #######
    query5 = """
    MATCH (g:Genre)<-[r:BELONGS_TO]-(c:CD)
    MATCH (c)-[r2:CONTAINS]->(t:Album)
    WHERE g.genre = $genre
    WITH t 
    LIMIT $limit
    MATCH (u:User)
    WHERE u.id = $user_id
    MERGE (u)-[r3:LIKES]->(t)
    """

    with driver.session(database = database_name) as session:
        session.run(query1, user_id = user_id)
        session.run(query2, user_id = user_id)
        session.run(query3, user_id = user_id, genre = genre, limit = limit)
        session.run(query4, user_id = user_id, genre = genre, limit = limit)
        session.run(query5, user_id = user_id, genre = genre, limit = limit)
    

**Adding some Likes to model User Preference:**

In [54]:
# User 1 likes "Rock" music
createExampleUser(user_id = 1, genre = "rock", limit = 50)
# User 2 likes "classic" music
# createExampleUser(user_id = 2, genre = "classic", limit = 5)
# User 3 likes "pop" music
# createExampleUser(user_id = 3, genre = "pop", limit = 50)
# User 4 likes "hip-hop" music
# createExampleUser(user_id = 4, genre = "hip-hop", limit = 50)
# User 5 likes "hip-hop" music
# createExampleUser(user_id = 5, genre = "hard rock", limit = 50)

***

#### Implementing Search Engine

In [95]:
user_id = 1

##### Feature 1: User Preference

In [96]:
query = """
// DELETE EXISTING PROJECTION
CALL gds.graph.drop('searchdomain_preference', false) 
YIELD graphName 
RETURN graphName
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,graphName
0,searchdomain_preference


In [97]:
query = f"""
// CREATE NEW PROJECTION WITH SEARCH RELEVANT SUB GRAPH
CALL gds.graph.project.cypher(
  'searchdomain_preference',
  ' // Liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(n) 
    WHERE u.id = {user_id} 
        AND (n:Song OR n:Album OR n:Artist) 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000
    
    UNION
    
    // CDs linked to liked Artists, Albums and Songs
    MATCH (u:User)-[:LIKES]->(x)-[:APPEARED_ON]->(n:CD) 
    WHERE u.id = {user_id} 
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 100000',
    
    'MATCH (u:User)-[:LIKES]->(n)
    WHERE u.id = {user_id} 
    AND (n:CD OR n:Song OR n:Album OR n:Artist) 
    MATCH (n)-[r:APPEARED_ON]->(m:CD) 
    RETURN id(n) AS source, id(m) AS target, type(r) AS type 
    LIMIT 100000' 
)
YIELD
  graphName, nodeCount AS nodes, relationshipCount AS rels
RETURN graphName, nodes, rels
"""

print(user_id)
#with driver.session(database = database_name) as session:
#    session.run(query)
        
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query, user_id = user_id)])
dtf_data

1


,graphName,nodes,rels
0,searchdomain_preference,2071,2108


In [98]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/
query = """
CALL gds.eigenvector.mutate('searchdomain_preference',  {
  mutateProperty: 'score_eig'
})
YIELD centralityDistribution, nodePropertiesWritten, ranIterations
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.000761,0.018063,2071


##### Feature 2: Content Match with Search Input

In [99]:
query = """
// DELETE EXISTING PROJECTION
CALL gds.graph.drop('searchdomain_content', false) 
YIELD graphName 
RETURN graphName
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data

,graphName
0,searchdomain_content


In [100]:
search_artist = "jimi hendrix"
search_album = "are you experienced"
search_song = "purple haze"

In [101]:
query = f"""
// CREATE NEW PROJECTION WITH SEARCH RELEVANT SUB GRAPH
CALL gds.graph.project.cypher(
  "searchdomain_content",
  
  " // Artists, Albums and Songs which match query
    MATCH (n) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*{search_artist}.*' 
    OR n.album  =~ '.*{search_album}.*' 
    OR n.song  =~ '.*{search_song}.*')
    RETURN id(n) AS id, labels(n) AS labels 
    LIMIT 1000
    
    UNION
    
    // CDS linked to Artists, Albums and Songs which match query
    MATCH (n)-[:APPEARED_ON]->(c:CD) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*{search_artist}.*' 
    OR n.album  =~ '.*{search_album}.*' 
    OR n.song  =~ '.*{search_song}.*')
    RETURN id(c) AS id, labels(c) AS labels 
    LIMIT 1000",
    
    "MATCH (n)-[r:APPEARED_ON]->(c:CD) 
    WHERE 
    (n:Song OR n:Album OR n:Artist) 
    AND 
    (n.artist =~ '.*{search_artist}.*' 
    OR n.album  =~ '.*{search_album}.*' 
    OR n.song  =~ '.*{search_song}.*')
    RETURN id(n) AS source, id(c) AS target, type(r) AS type 
    LIMIT 1000"
)
YIELD
  graphName, nodeCount AS nodes, relationshipCount AS rels
RETURN graphName, nodes, rels
"""


dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)]) 
dtf_data

,graphName,nodes,rels
0,searchdomain_content,141,166


In [102]:
# https://neo4j.com/docs/graph-data-science/current/algorithms/eigenvector-centrality/
query = """
CALL gds.eigenvector.mutate('searchdomain_content',  {
  mutateProperty: 'score_eig'
})
YIELD centralityDistribution, nodePropertiesWritten, ranIterations
RETURN centralityDistribution.min AS minimumScore, centralityDistribution.mean AS meanScore, nodePropertiesWritten
"""
dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,minimumScore,meanScore,nodePropertiesWritten
0,0.002552,0.070646,141


Combined Search Results:

In [103]:
query = """
CALL {

    CALL {
        CALL gds.eigenvector.stream('searchdomain_content')
        YIELD nodeId, score
        WHERE gds.util.asNode(nodeId):CD
        WITH gds.util.asNode(nodeId).id AS nodeId, score as score_cont
        RETURN nodeId, score_cont
    }
    RETURN nodeId, score_cont, 0 as score_pref

    UNION 

    CALL {
        CALL gds.eigenvector.stream('searchdomain_preference')
        YIELD nodeId, score
        WHERE gds.util.asNode(nodeId):CD
        WITH gds.util.asNode(nodeId).id AS nodeId, score as score_pref
        RETURN nodeId, score_pref
    }
    RETURN nodeId, 0 as score_cont, score_pref

}

WITH DISTINCT nodeId, count(*) as count, sum(score_cont) as score_cont, sum(score_pref) as score_pref

MATCH (n:CD)
WHERE n.id = nodeId
OPTIONAL MATCH (n)-[:CONTAINS]->(ar:Artist)
OPTIONAL MATCH (n)-[:CONTAINS]->(ab:Album)
OPTIONAL MATCH (n)-[:CONTAINS]->(so:Song)

RETURN nodeId, 
count,
score_cont, score_pref,
collect(DISTINCT ar.artist) as artists,
collect(DISTINCT ab.album) as albums, 
collect(DISTINCT so.song) as songs


ORDER BY score_cont DESC, score_pref DESC
LIMIT 20
"""

dtf_data = pd.DataFrame([dict(_) for _ in session.run(query)])
dtf_data.head(30)

,nodeId,count,score_cont,score_pref,artists,albums,songs
0,7923,1,0.291747,0.000000,[signature licks],[jimi hendrix],"[fire (verse);, foxey lady (solo, slow);, fire..."
1,35734,2,0.176069,0.034758,[jimi hendrix],[are you experienced],"[i don't live today, 51st anniversary, stone f..."
2,677,2,0.176069,0.034758,[jimi hendrix],[are you experienced],"[third stone from the sun, remember, fire, can..."
3,136907,2,0.176069,0.034758,[jimi hendrix],[are you experienced],"[love or confusion, remember, highway chile, m..."
4,33321,2,0.176069,0.017759,[jimi hendrix],[astro man(alchemy); - studio outtakes 1966-68],"[purple haze 1 (4);, 51st anniversary (5);, la..."
5,30138,2,0.176069,0.017759,[jimi hendrix],[astro man box set],"[can you see me (pre unre);, purple haze (pre ..."
6,20709,1,0.176069,0.000000,[jimi hendrix experience],[the vpro archive - it must be dusty],"[mockingbird (mpeg1 video data);, spoken word,..."
7,2023,2,0.118230,0.034758,[jimi hendrix],[experience hendrix - the best of jimi hendrix],"[bold as love, night bird flying, if 6 was 9, ..."
8,1936,2,0.118230,0.034758,[jimi hendrix],[experience hendrix],"[bold as love, night bird flying, if 6 was 9, ..."
9,17266,2,0.118230,0.034758,[jimi hendrix],[are you experienced],"[3rd stone from the sun, remember, foxy lady, ..."
